In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium_stealth import stealth
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.proxy import Proxy, ProxyType

print('Done')

Done


# Download and parse price data from TTC

In [3]:
!curl https://eu.tamrieltradecentre.com/download/PriceTable --output "PriceTable.zip"
!unzip -o PriceTable.zip -d PriceTable/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2693k  100 2693k    0     0   530k      0  0:00:05  0:00:05 --:--:--  599k
Archive:  PriceTable.zip
  inflating: PriceTable/ItemLookUpTable_DE.lua  
  inflating: PriceTable/ItemLookUpTable_EN.lua  
  inflating: PriceTable/ItemLookUpTable_ES.lua  
  inflating: PriceTable/ItemLookUpTable_FR.lua  
  inflating: PriceTable/ItemLookUpTable_RU.lua  
  inflating: PriceTable/ItemLookUpTable_ZH.lua  
  inflating: PriceTable/PriceTableEU.lua  


In [4]:
import json
from slpp import slpp as lua

with open('PriceTable/PriceTableEU.lua', 'r') as file:
    data = file.read()
    data = data.split("self.PriceTable=")[1][:-5]
    data = lua.decode(data)['Data']
    json.dump(data, open(f"PriceTable.json", "w"), indent=4)

with open('PriceTable/ItemLookUpTable_EN.lua', 'r') as file:
    data = file.read()
    data = data.split("self.ItemLookUpTable=")[1][:-5]
    data = lua.decode(data)
    json.dump(data, open(f"ItemLookUpTable.json", "w"), indent=4)

print('Done')

Done


# Parse listed items from TTC by query

In [5]:
def get_driver(proxy=None):
    options = webdriver.ChromeOptions()
    options.add_argument("start-maximized")
    options.add_argument("--headless")
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)

    capabilities = webdriver.DesiredCapabilities.CHROME

    if proxy is not None:
        prox = Proxy()
        prox.proxy_type = ProxyType.MANUAL
        prox.http_proxy = proxy
        prox.add_to_capabilities(capabilities)

    driver = webdriver.Chrome(ChromeDriverManager().install(),
                              options=options,
                              desired_capabilities=capabilities)

    stealth(driver,
            languages=["en-US", "en"],
            vendor="Google Inc.",
            platform="Win32",
            webgl_vendor="Intel Inc.",
            renderer="Intel Iris OpenGL Engine",
            fix_hairline=True,
            )

    return driver

def get_page_rows(driver, url, page):
    driver.get(f"{url}&page={page}")
    driver.implicitly_wait(4)
    return driver.find_elements(By.XPATH, '//tr[@class="cursor-pointer"]')

def parse_row_data(row_html):
    d = {
        'name': row_html.find_element(By.XPATH, './/td[1]/div[1]').text,
        'location': row_html.find_element(By.XPATH, './/td[3]/div[1]').text,
        'guild': row_html.find_element(By.XPATH, './/td[3]/div[2]').text,
        'price': row_html.find_element(By.XPATH, './/td[4]/span[1]').text,
        'quality': row_html.find_element(By.XPATH, './/td[1]/div[1]').get_attribute("class"),
        'cp': row_html.find_element(By.XPATH, './/td[1]/div[2]/img').get_attribute("src"),
        'level': row_html.find_element(By.XPATH, './/td[1]/div[2]/span').text,
        'last_seen': row_html.find_element(By.XPATH, './/td[5]').text,
    }
    d['quality'] = class_to_quality[d['quality']]
    d['price'] = float(d['price'].replace(',', ''))
    d['cp'] = 'championPoint' in d['cp']
    if d['cp']:
        d['level'] = int(d['level']) + 50
    return d

def get_row_data(url, page, driver=None):
    if driver is None:
        driver = get_driver()

    page_rows = get_page_rows(driver, url, page)
    dict_rows = [parse_row_data(r) for r in page_rows]
    return dict_rows


In [6]:
class_to_quality = {
    'item-quality-normal': 0,
    'item-quality-fine': 1,
    'item-quality-superior': 2,
    'item-quality-epic': 3,
    'item-quality-legendary': 4
}


# Input your params here
- What are you searching for (use + instead of space, no url encoding yet)
- how many pages you want to search
- other TTC URL params

In [7]:
# params
n_pages = 3
params = {
    'ItemNamePattern': 'motif',
    'ItemQualityID': 3,
    'PriceMin': 3000,
    'PriceMax': 50000,
    'SortBy': 'Price',
    'Order': 'asc'
}

In [8]:
base_url = "https://eu.tamrieltradecentre.com"
search_url = f"{base_url}/{'pc/Trade/SearchResult?'}"
param_str = '&'.join([f"{k}={v}" for k, v in params.items()])
url = f"{search_url}{param_str}"
url

'https://eu.tamrieltradecentre.com/pc/Trade/SearchResult?ItemNamePattern=motif&ItemQualityID=3&PriceMin=3000&PriceMax=50000&SortBy=Price&Order=asc'

In [9]:
driver = get_driver()



====== WebDriver manager ======
Current google-chrome version is 110.0.5481
Get LATEST chromedriver version for 110.0.5481 google-chrome
Driver [/Users/tartakovsky/.wdm/drivers/chromedriver/mac64/110.0.5481.77/chromedriver] found in cache
/var/folders/04/60qvc12j259_3sx6n2hxnldc0000gn/T/ipykernel_39756/698962434.py:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(),


In [10]:
rows = []

for page in range(1, 5001):
    dict_rows = get_row_data(url, page, driver)
    rows = rows + dict_rows
    print(len(rows))
    if len(dict_rows) == 0 or page == n_pages:
        break
    #
    # driver.implicitly_wait(1)

len(rows)

10
20
30


30

In [11]:
rows

[{'name': 'Crafting Motif 113: Steadfast Society Maces',
  'location': 'Summerset: Shimmerene',
  'guild': 'Arkays Handelshof',
  'price': 3000.0,
  'quality': 3,
  'cp': False,
  'level': '1',
  'last_seen': '2 Hour ago'},
 {'name': 'Crafting Motif 113: Steadfast Society Bows',
  'location': 'Summerset: Shimmerene',
  'guild': 'Arkays Handelshof',
  'price': 3000.0,
  'quality': 3,
  'cp': False,
  'level': '1',
  'last_seen': '2 Hour ago'},
 {'name': 'Crafting Motif 81: New Moon Priest Maces',
  'location': 'High Isle and Amenos: Gonfalon Bay',
  'guild': 'Dragonborn Inc',
  'price': 3000.0,
  'quality': 3,
  'cp': False,
  'level': '1',
  'last_seen': '3 Hour ago'},
 {'name': 'Crafting Motif 113: Steadfast Society Legs',
  'location': 'Summerset: Shimmerene',
  'guild': 'Arkays Handelshof',
  'price': 3000.0,
  'quality': 3,
  'cp': False,
  'level': '1',
  'last_seen': '4 Hour ago'},
 {'name': 'Crafting Motif 81: New Moon Priest Helmets',
  'location': 'The Reach: Markarth',
  'gui

In [12]:
lookup = json.load(open(f"ItemLookUpTable.json", "r"))
prices = json.load(open(f"PriceTable.json", "r"))
lookup

{'sip of weapon power': {'450': 1},
 'dwarven cuirass of stamina': {'300': 2},
 'gingerose tea': {'21': 4},
 'slight glyph of increase physical harm': {'1250': 6},
 'superb glyph of decrease health': {'950': 7},
 'voidsteel dagger of shock': {'250': 8},
 'hide jack of stamina': {'300': 9},
 'darkstride jack': {'300': 10},
 'calcinium battle axe of shock': {'250': 11},
 'greaves of the fire': {'300': 12},
 'fell jack of stamina': {'300': 13},
 'dram of health': {'450': 14},
 'dwarven girdle of health': {'300': 15},
 'mahogany inferno staff': {'250': 16},
 "oath-speaker's boots": {'300': 17},
 'cuirass of the fang': {'300': 18},
 "huntmaster's leash": [19],
 'happy ogrim amber ale': {'24': 20},
 'bitter remorse ale': {'23': 21},
 'recipe: ginkgo tonic': {'171': 23},
 "whitestrake's hat": {'300': 24},
 'major glyph of flame': {'950': 25},
 'dunerunners': {'300': 26},
 'recipe: hello handsome porter': {'171': 27},
 'yew inferno staff of flame': {'250': 28},
 "pact's sword": {'250': 29},
 '

In [13]:
for row in rows:
    l = lookup[row['name'].lower()]
    row['ttc_item_id'] = list(l.values())[0]
    row['ttc_price'] = prices[f"{row['ttc_item_id']}"][f"{row['quality']}"][f"{row['level']}"]
    if isinstance(row['ttc_price'], dict):
        row['ttc_price'] = list(row['ttc_price'].values())
    row['ttc_price'] = [p.get('SuggestedPrice', None) for p in row['ttc_price']]
    row['ttc_suggested'] = [p for p in row['ttc_price'] if p is not None]

    if len(row['ttc_suggested']) == 0:
        row['ttc_suggested'] = None
    else:
        row['ttc_suggested'] = min(row['ttc_suggested'])
    row['price_ratio'] = row['ttc_suggested'] / row['price']
rows

[{'name': 'Crafting Motif 113: Steadfast Society Maces',
  'location': 'Summerset: Shimmerene',
  'guild': 'Arkays Handelshof',
  'price': 3000.0,
  'quality': 3,
  'cp': False,
  'level': '1',
  'last_seen': '2 Hour ago',
  'ttc_item_id': 25192,
  'ttc_price': [1378.86],
  'ttc_suggested': 1378.86,
  'price_ratio': 0.45962},
 {'name': 'Crafting Motif 113: Steadfast Society Bows',
  'location': 'Summerset: Shimmerene',
  'guild': 'Arkays Handelshof',
  'price': 3000.0,
  'quality': 3,
  'cp': False,
  'level': '1',
  'last_seen': '2 Hour ago',
  'ttc_item_id': 25229,
  'ttc_price': [1634.57],
  'ttc_suggested': 1634.57,
  'price_ratio': 0.5448566666666667},
 {'name': 'Crafting Motif 81: New Moon Priest Maces',
  'location': 'High Isle and Amenos: Gonfalon Bay',
  'guild': 'Dragonborn Inc',
  'price': 3000.0,
  'quality': 3,
  'cp': False,
  'level': '1',
  'last_seen': '3 Hour ago',
  'ttc_item_id': 21493,
  'ttc_price': [396.2],
  'ttc_suggested': 396.2,
  'price_ratio': 0.13206666666

In [14]:
len([r for r in rows if r['price_ratio'] > 1.05])

0

In [15]:
json.dump(rows, open('dump.json', 'w'), indent=4)